### 1. 문제 정의
- 사내 이직률을 낮춰 다니고 싶은 회사, 떠나기 싫은 회사 이미지를 만들기

### 2. 데이터 수집
- 직원 이직 데이터

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import pandas as pd
df = pd.read_csv('data/job_transfer.csv')

In [11]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


### 3. 데이터 전처리

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

### 4. 탐색적 데이터 분석

In [14]:
# 이직
df['Attrition'].value_counts()

Attrition
No     1233
Yes     237
Name: count, dtype: int64

In [15]:
# 이직 = 0, 이직 x = 1로 수정
import numpy as np
df['Attrition'] = np.where(df['Attrition']=='Yes', 1, 0)
df['Attrition'].value_counts()

Attrition
0    1233
1     237
Name: count, dtype: int64

In [16]:
# 성별, 연령 분석
# -> 구간화
np.where(df['Age'] <= 30, '30이하', np.where(df['Age'] <= 40, '30~40대', '40대 이상'))


array(['40대 이상', '40대 이상', '30~40대', ..., '30이하', '40대 이상', '30~40대'],
      dtype='<U6')

In [17]:
df['AgeGroup'] = np.where(df['Age'] <= 30, '30이하', np.where(df['Age'] <= 40, '30~40대', '40대 이상'))

In [18]:
df['AgeGroup'].value_counts()

AgeGroup
30~40대    619
40대 이상    465
30이하      386
Name: count, dtype: int64

In [22]:
# 연령 그룹별 이직률 현황
df_gp = df.groupby('AgeGroup')['Attrition'].agg(['sum', 'count'])
df_gp['rate'] = round((df_gp['sum'] / df_gp['count'])*100,1)
df_gp

,sum,count,rate
AgeGroup,,,
30~40대,85,619,13.7
30이하,100,386,25.9
40대 이상,52,465,11.2


In [24]:
# 성별 이직률 현황
df_gp2 = df.groupby('Gender')['Attrition'].agg(['sum', 'count'])
df_gp2['rate'] = round((df_gp2['sum'] / df_gp2['count'])*100,1)
df_gp2
# 남성 직원의 이직률이 좀 더 높음

,sum,count,rate
Gender,,,
Female,87,588,14.8
Male,150,882,17.0


In [25]:
# 부서별 이직률 현황
df_gp3 = df.groupby('Department')['Attrition'].agg(['sum', 'count'])
df_gp3['rate'] = round((df_gp3['sum'] / df_gp3['count'])*100,1)
df_gp3

,sum,count,rate
Department,,,
Human Resources,12,63,19.0
Research & Development,133,961,13.8
Sales,92,446,20.6
